In [205]:
# !pip install datasets
# !pip install Pillow
# !pip install numpy
# !pip install matplotlib
# !pip install import_ipynb
# !pip install tensorflow
# !pip install tensorflow.keras

In [206]:
from google.colab import userdata

# Retrieve the GitHub token from userdata
myGithub = userdata.get('githubKey')

!git clone https://{myGithub}@github.com/fitristachan/invoice-image-processing.git /content/invoice-image-processing
!git -C invoice-image-processing pull

fatal: destination path '/content/invoice-image-processing' already exists and is not an empty directory.
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (1/1), 187 bytes | 187.00 KiB/s, done.
From https://github.com/fitristachan/invoice-image-processing
   8c1d80f..e3c86be  main       -> origin/main
Updating 8c1d80f..e3c86be
Fast-forward


In [207]:
import sys
import os

sys.path.append('/content/invoice-image-processing')
print(os.listdir('/content/invoice-image-processing'))

!jupyter nbconvert --to script '/content/invoice-image-processing/data_etl_invoice.ipynb' --output data_etl

['invoice-image-processing.code-workspace', '__pycache__', 'requirements.txt', 'preprocessing.ipynb', '.git', 'data_etl_invoice.ipynb', 'Dataset', 'data_etl.py', 'model_ver_1.ipynb']
[NbConvertApp] Converting notebook /content/invoice-image-processing/data_etl_invoice.ipynb to script
[NbConvertApp] Writing 6902 bytes to /content/invoice-image-processing/data_etl.py


In [208]:
# import import_ipynb
import data_etl
from data_etl import DatasetReceipt

In [209]:
# !pip install torch
# !pip install torchvision
# !pip install scikit-image
# !pip install scipy
# !pip install easyocr

In [210]:
from datasets import load_dataset
import easyocr
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

**Text Extraction**

> using easyocr

> support bahasa indonesia and english





In [211]:
reader = easyocr.Reader(['en', 'id'])

def is_receipt(text, logic='AND'):
    required_keywords = ['total', 'amount', 'jumlah', 'price', 'harga']
    text_lower = text.lower()

    #diubah nanti jadinya ada yang and ada yang or
    if logic == 'AND':
        return all(keyword in text_lower for keyword in required_keywords)
    elif logic == 'OR':
        return any(keyword in text_lower for keyword in required_keywords)
    else:
        raise ValueError("Invalid logic parameter. Use 'AND' or 'OR'.")


def process_image(image, logic='AND'):
    # Convert PIL Image to numpy array
    image_np = np.array(image)

    # Extract text using EasyOCR
    results = reader.readtext(image_np)

    # Combine all detected text into a single string
    extracted_text = ' '.join([result[1] for result in results])

    # Check if the extracted text is a receipt
    if is_receipt(extracted_text, logic=logic):
        print("This is a valid receipt.")
        print("Extracted Text:")
        print(extracted_text)
    else:
        print("This is not a valid receipt.")

Training

In [212]:
# Initialize dataset
dataset = DatasetReceipt(dataset_name="naver-clova-ix/cord-v2", split="train")

# Verify dataset
for i in range(5):
    sample = dataset[i]
    print(f"Sample {i}:")
    print("Image Shape:", sample["image"].shape)
    print("Label:", sample["label"])
    print("-" * 50)

# Create DataLoader
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Verify DataLoader
for batch in dataloader:
    print("Batch Images Shape:", batch["image"].shape)
    print("Batch Labels:", batch["label"])
    break  # Print only the first batch

Sample 0:
Image Shape: torch.Size([1, 600, 600])
Label: -1
--------------------------------------------------
Sample 1:
Image Shape: torch.Size([1, 600, 600])
Label: -1
--------------------------------------------------
Sample 2:
Image Shape: torch.Size([1, 600, 600])
Label: -1
--------------------------------------------------
Sample 3:
Image Shape: torch.Size([1, 600, 600])
Label: -1
--------------------------------------------------
Sample 4:
Image Shape: torch.Size([1, 600, 600])
Label: -1
--------------------------------------------------
Batch Images Shape: torch.Size([8, 1, 600, 600])
Batch Labels: tensor([-1, -1, -1, -1, -1, -1, -1, -1])


In [213]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=5000, output_dim=128),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(64),
#     tf.keras.layers.Dense(32, activation="relu"),
#     tf.keras.layers.Dense(3, activation="softmax")  # 3 kelas: menu, qty, price
# ])

# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [214]:
# model.fit(padded, labels, epochs=10, batch_size=32)

In [215]:
# # Gunakan dengan DataLoader untuk training
# from torch.utils.data import DataLoader

# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# # Cek satu batch
# for batch in dataloader:
#     print(batch["image"].shape)  # Harusnya (32, 1, 600, 600)
#     break

In [216]:
# # Load the CORD-v2 dataset
# dataset = load_dataset("naver-clova-ix/cord-v2")

# # Example: Process the first few images in the dataset
# for example in dataset['train'].select(range(5)):  # Process the first 5 examples
#     image = example['image']  # Get the PIL Image object
#     print("Processing image...")

#     # Process the image with OR logic
#     process_image(image, logic='OR')
#     print("-" * 50)

# # Load international style dataset
# dataset_international = load_dataset("katanaml-org/invoices-donut-data-v1")

# # Example: Process the first few images in the dataset_international
# for example_int in dataset_international['train'].select(range(5)):  # Process the first 5 examples
#     image_int = example_int['image']
#     print("Processing image international...")

#     # Process the image with OR logic
#     process_image(image_int, logic='OR')
#     print("-" * 50)